In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time

In [9]:
chromedriver = './chromedriver'
def get_web_page2(url):
    from selenium import webdriver
    driver = webdriver.Chrome(chromedriver) 
    driver.get(url)
    time.sleep(3) 
    html = driver.page_source 
    driver.close() 
    
    return(BeautifulSoup(html,'html.parser'))

In [10]:
def detail_page_sale(url):
    soup = get_web_page2(url)
    obj = soup.find_all('div',attrs={'class':'j-house houseList-item clearfix z-hastag'})
    
    result = pd.DataFrame(columns=['head', 'age', 'section', 'addr', 'shape', 'price', 'unit_price', 'floor', 'layout', 
                                   'area', 'main_area', 'community','link'])
    for oo in obj:
        head = oo.find('div',attrs={'class':'houseList-item-title'}).a.contents[0]
        link = 'https://sale.591.com.tw' + oo.find('div',attrs={'class':'houseList-item-title'}).a['href']
        price = int(''.join(oo.find('div',attrs={'class':'houseList-item-price'}).em.contents[0].split(',')))
        unit_price = oo.find('div',attrs={'class':'houseList-item-unitprice'}).contents[0]
        
        shape = np.nan; layout = np.nan; area = np.nan; main_area = np.nan; age = np.nan; floor=np.nan;
        community = np.nan; section = np.nan; addr = np.nan;
        
        tmp1 = oo.find_all('div',attrs={'class':'houseList-item-attr-row clearfix'})
        for t in tmp1:
            if t.find('span', attrs={'class':'houseList-item-attrs-shape'}) is not None:
                shape = t.find('span', attrs={'class':'houseList-item-attrs-shape'}).contents[0]
            if t.find('span', attrs={'class':'houseList-item-attrs-layout'}) is not None:
                layout = t.find('span', attrs={'class':'houseList-item-attrs-layout'}).contents[0]
            if t.find('span', attrs={'class':'houseList-item-attrs-area'}) is not None:
                area = t.find('span', attrs={'class':'houseList-item-attrs-area'}).contents[0]
            if t.find('span', attrs={'class':"houseList-item-attrs-mainarea"}) is not None:
                main_area = t.find('span', attrs={'class':'houseList-item-attrs-mainarea'}).contents[0]
            if t.find('span', attrs={'class':'houseList-item-attrs-houseage'}) is not None:
                age = t.find('span', attrs={'class':'houseList-item-attrs-houseage'}).contents[0]
            if t.find('span', attrs={'class':'houseList-item-attrs-floor'}) is not None:
                floor = t.find('span', attrs={'class':'houseList-item-attrs-floor'}).contents[0]
        
        tmp2 = oo.find_all('div',attrs={'class':'houseList-item-address-row clearfix'})
        for t2 in tmp2:
            if t2.find('span', attrs={'class':'houseList-item-community mr15 pull-left'}) is not None:
                if t2.find('span', attrs={'class':'houseList-item-community mr15 pull-left'}).a is not None:
                    community = t2.find('span', attrs={'class':'houseList-item-community mr15 pull-left'}).a.contents[0]
            if t2.find('span', attrs={'class':'houseList-item-section pull-left'}) is not None:
                section = t2.find('span', attrs={'class':'houseList-item-section pull-left'}).contents[0]
            if t2.find('span', attrs={'class':'houseList-item-address pull-left'}) is not None:
                addr = t2.find('span', attrs={'class':'houseList-item-address pull-left'}).contents[0]
        
        result = pd.concat([result, 
                            pd.DataFrame([[head, age, section, addr, shape, price, unit_price, floor, layout, area, 
                                           main_area, community, link]],
                            columns=['head', 'age', 'section', 'addr', 'shape', 'price', 'unit_price', 'floor', 'layout', 
                                     'area', 'main_area', 'community','link'])],0)
    return(result.reset_index(drop=True))

In [11]:
def crawler_main2(url):
    soup = get_web_page2(url)
    ttlnum = int(soup.find('a', attrs={'class':'pageNum-form'})['data-total'])
    ttlpage = int(ttlnum/30)+1
    
    result = pd.DataFrame(columns=['head', 'age', 'section', 'addr', 'shape', 'price', 'unit_price', 'floor', 'layout', 
                                   'area', 'main_area', 'community','link'])
    
    print('Processing Pages ({} Pages In Total): '.format(ttlpage))
    for l in range(ttlpage):
        print('{}'.format(l), end='\t')
        url2 = url + '&firstRow={}'.format(l*30)
        result = detail_page_sale(url2)
        #result = pd.concat([result, tmp], 0)
        result.to_csv('tpe_v2.csv',index=False, mode='a')
        #time.sleep(1)
        
    
    print('\nProcess Done!!\n')
    return(result.reset_index(drop=True))

#### Taipei Sale Data

In [ ]:
url = 'https://sale.591.com.tw/?shType=list&regionid=1'
taipei_sale = crawler_main2(url)